In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import pandas as pd
import numpy as np

/Users/aryan/Desktop/Umich/Terms/F24/SI650/Project/satire-news-search-engine/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('../data/satirical_news_articles.csv')

In [3]:
df.columns

Index(['headline', 'body', 'website', 'url', 'image_link'], dtype='object')

In [4]:
print(df.body[0])

LOS ANGELES—Admitting that she’d made some major lifestyle changes since developing her famous KonMari method, a relaxed Marie Kondo told reporters Tuesday that she was now perfectly happy living in waist-high sewage. “The truth is, while I used to be very hard on myself about keeping everything clean, I’m now able to find peace living my life half-submerged in a large, fetid pool of human waste,” said Kondo, who added that while things like tidiness, organization, and minimalism used to spark joy for her, she now felt that same warmth from wading, floating, and swimming in the many gallons of untreated urine and feces that currently filled her home. “It was difficult, but once I had my children, I began to find it impossible to remove, clean, and sanitize the unending stream of excrement that bubbled up out of my toilets and filled my house to the point of collapse. While I used to hate it, I now see that each individual piece of human shit plays a part in creating a relaxed, easygoin

In [5]:
tokenizer = T5Tokenizer.from_pretrained('BeIR/query-gen-msmarco-t5-large-v1')
model = T5ForConditionalGeneration.from_pretrained('BeIR/query-gen-msmarco-t5-large-v1')

para = df.body[0]

input_ids = tokenizer.encode(para, return_tensors='pt')
outputs = model.generate(
    input_ids=input_ids,
    max_length=64,
    do_sample=True,
    top_p=0.95,
    num_return_sequences=3)

print("Paragraph:")
print(para)

print("\nGenerated Queries:")
for i in range(len(outputs)):
    query = tokenizer.decode(outputs[i], skip_special_tokens=True)
    print(f'{i + 1}: {query}')

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Paragraph:
LOS ANGELES—Admitting that she’d made some major lifestyle changes since developing her famous KonMari method, a relaxed Marie Kondo told reporters Tuesday that she was now perfectly happy living in waist-high sewage. “The truth is, while I used to be very hard on myself about keeping everything clean, I’m now able to find peace living my life half-submerged in a large, fetid pool of human waste,” said Kondo, who added that while things like tidiness, organization, and minimalism used to spark joy for her, she now felt that same warmth from wading, floating, and swimming in the many gallons of untreated urine and feces that currently filled her home. “It was difficult, but once I had my children, I began to find it impossible to remove, clean, and sanitize the unending stream of excrement that bubbled up out of my toilets and filled my house to the point of collapse. While I used to hate it, I now see that each individual piece of human shit plays a part in creating a relaxe

In [6]:
# Drop unnecessary columns
df = df.drop(['website', 'url', 'image_link'], axis=1)

# Generate random 8 digit non-repeating numbers for docids
n_docs = len(df)
docids = np.random.choice(range(10000000, 100000000), size=n_docs, replace=False)
df['docid'] = docids

# Reorder columns to have docid first
df = df[['docid', 'headline', 'body']]

In [7]:
df.to_csv('../data/processed_articles.csv', index=False)